In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE123088"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE123088"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE123088.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE123088.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE123088.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this appears to be a gene expression dataset from CD4+ T cells
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Asthma), row 1 contains 'primary diagnosis' which includes 'ASTHMA'
trait_row = 1

# For gender, row 2 and 3 contain 'Sex: Male' and 'Sex: Female'
gender_row = 2  # This row seems to have more gender entries

# For age, row 3 and 4 contain age information
age_row = 3  # This row seems to have more age entries

# 2.2 Data Type Conversion
def convert_trait(value):
    if not isinstance(value, str):
        return None
    value = value.lower()
    if 'diagnosis' not in value:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'asthma' in value.lower():
        return 1
    else:
        return 0

def convert_gender(value):
    if not isinstance(value, str):
        return None
    if 'sex' not in value.lower():
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

def convert_age(value):
    if not isinstance(value, str):
        return None
    if 'age' not in value.lower():
        return None
    if ':' in value:
        try:
            age = int(value.split(':', 1)[1].strip())
            return age
        except:
            return None
    return None

# 3. Save Metadata - initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create DataFrame from the sample characteristics dictionary
    sample_characteristics_dict = {0: ['cell type: CD4+ T cells'], 
                                  1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 
                                      'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 
                                      'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 
                                      'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 
                                      'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 
                                      'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 
                                  2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 
                                      'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 
                                      'diagnosis2: OBESITY'], 
                                  3: ['age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 
                                      'age: 80', 'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 
                                      'age: 81', 'age: 70', 'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 
                                      'age: 45', 'age: 65', 'age: 48', 'age: 50', 'age: 24', 'age: 42'], 
                                  4: [float('nan'), 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 'age: 16', 
                                      'age: 12', 'age: 27']}
    
    clinical_data = pd.DataFrame.from_dict(sample_characteristics_dict, orient='index')
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist and save the clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{0: [1.0, 56.0, 1.0], 1: [0.0, nan, nan], 2: [0.0, 20.0, 0.0], 3: [0.0, 51.0, nan], 4: [0.0, 37.0, nan], 5: [0.0, 61.0, nan], 6: [0.0, 31.0, nan], 7: [0.0, 41.0, nan], 8: [0.0, 80.0, nan], 9: [0.0, 53.0, nan], 10: [0.0, 73.0, nan], 11: [0.0, 60.0, nan], 12: [0.0, 76.0, nan], 13: [0.0, 77.0, nan], 14: [0.0, 74.0, nan], 15: [0.0, 69.0, nan], 16: [nan, 81.0, nan], 17: [nan, 70.0, nan], 18: [nan, 82.0, nan], 19: [nan, 67.0, nan], 20: [nan, 78.0, nan], 21: [nan, 72.0, nan], 22: [nan, 66.0, nan], 23: [nan, 36.0, nan], 24: [nan, 45.0, nan], 25: [nan, 65.0, nan], 26: [nan, 48.0, nan], 27: [nan, 50.0, nan], 28: [nan, 24.0, nan], 29: [nan, 42.0, nan]}
Clinical data saved to ../../output/preprocess/Asthma/clinical_data/GSE123088.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE123088/GSE123088_series_matrix.txt.gz


Gene data shape: (24166, 204)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers shown are not standard human gene symbols
# They appear to be numeric indices or probe IDs that would need mapping to actual gene symbols
# Standard human gene symbols would typically be formatted like BRCA1, TP53, etc.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# Check if there are any platforms defined in the SOFT file that might contain annotation data
with gzip.open(soft_file, 'rt') as f:
    soft_content = f.read()

# Look for platform sections in the SOFT file
platform_sections = re.findall(r'^!Platform_title\s*=\s*(.+)$', soft_content, re.MULTILINE)
if platform_sections:
    print(f"Platform title found: {platform_sections[0]}")

# Try to extract more annotation data by reading directly from the SOFT file
# Look for lines that might contain gene symbol mappings
symbol_pattern = re.compile(r'ID_REF\s+Symbol|ID\s+Gene Symbol', re.IGNORECASE)
annotation_lines = []
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if symbol_pattern.search(line):
            annotation_lines.append(line)
            # Collect the next few lines to see the annotation structure
            for _ in range(10):
                annotation_lines.append(next(f, ''))

if annotation_lines:
    print("Found potential gene symbol mappings:")
    for line in annotation_lines:
        print(line.strip())

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("\nGene annotation preview:")
print(preview_df(gene_annotation, n=10))

# If we need an alternative source of mapping, check if there are any other annotation files in the cohort directory
cohort_files = os.listdir(in_cohort_dir)
annotation_files = [f for f in cohort_files if 'annotation' in f.lower() or 'platform' in f.lower()]
if annotation_files:
    print("\nAdditional annotation files found in the cohort directory:")
    for file in annotation_files:
        print(file)


Platform title found: Agilent-039494 SurePrint G3 Human GE v2 8x60K Microarray 039381 (Entrez Gene ID  version)



Gene annotation preview:
{'ID': ['1', '2', '3', '9', '10', '12', '13', '14', '15', '16'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10', '12', '13', '14', '15', '16'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# From the previous output, I can see:
# - The gene expression data uses numeric IDs as identifiers (e.g., '1', '2', '3')
# - The gene annotation contains columns 'ID' and 'ENTREZ_GENE_ID'
# - The annotation shows that 'ID' column contains the same identifiers seen in gene expression data
# - 'ENTREZ_GENE_ID' contains gene IDs that we can use to map to gene symbols

# First check if we have any additional mapping resources
mapping_id_column = 'ID'  # This matches the identifiers in gene_data
mapping_gene_column = 'ENTREZ_GENE_ID'  # We'll use this as the gene identifier

print(f"Using '{mapping_id_column}' to map to '{mapping_gene_column}'")

# 2. Extract the mapping data
mapping_data = get_gene_mapping(gene_annotation, mapping_id_column, mapping_gene_column)
print(f"Mapping data shape: {mapping_data.shape}")
print("First few rows of mapping data:")
print(mapping_data.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())

# Save the mapped gene expression data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

Using 'ID' to map to 'ENTREZ_GENE_ID'


Mapping data shape: (4740924, 2)
First few rows of mapping data:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10


Mapped gene expression data shape: (0, 204)
First few rows of gene expression data:
Empty DataFrame
Columns: [GSM3494884, GSM3494885, GSM3494886, GSM3494887, GSM3494888, GSM3494889, GSM3494890, GSM3494891, GSM3494892, GSM3494893, GSM3494894, GSM3494895, GSM3494896, GSM3494897, GSM3494898, GSM3494899, GSM3494900, GSM3494901, GSM3494902, GSM3494903, GSM3494904, GSM3494905, GSM3494906, GSM3494907, GSM3494908, GSM3494909, GSM3494910, GSM3494911, GSM3494912, GSM3494913, GSM3494914, GSM3494915, GSM3494916, GSM3494917, GSM3494918, GSM3494919, GSM3494920, GSM3494921, GSM3494922, GSM3494923, GSM3494924, GSM3494925, GSM3494926, GSM3494927, GSM3494928, GSM3494929, GSM3494930, GSM3494931, GSM3494932, GSM3494933, GSM3494934, GSM3494935, GSM3494936, GSM3494937, GSM3494938, GSM3494939, GSM3494940, GSM3494941, GSM3494942, GSM3494943, GSM3494944, GSM3494945, GSM3494946, GSM3494947, GSM3494948, GSM3494949, GSM3494950, GSM3494951, GSM3494952, GSM3494953, GSM3494954, GSM3494955, GSM3494956, GSM3494957, GS